In [1]:
import re
import transformers
import pandas as pd
from wordcloud import STOPWORDS
from cleantext import clean

/Users/sreevaatsav/.pyenv/versions/project_env/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# https://www.kaggle.com/code/colearninglounge/nlp-data-preprocessing-and-cleaning

def remove_hashtags(data):
    clean_data = re.sub(r"#[A-Za-z0-9_]+", "", data)
    clean_data = re.sub(r"\$[A-Za-z0-9_]+", "", clean_data)
    return clean_data

def remove_hashsymbol(data):
    clean_data = re.sub(r"#", "", data)
    return clean_data

def remove_punctuations(data):
    punct_tag=re.compile(r'[^\w\s]')
    data=punct_tag.sub(r'',data)
    return data

def remove_comma(data):
    punct_tag=re.compile(r'[^\w\s]')
    data=punct_tag.sub(r' ',data)
    return data

def remove_html(data):
    data = re.sub(r"https?://[^\s]+", "", data)
    
    return data

def remove_url(data):
    url_clean= re.compile(r"https://\S+|www\.\S+")
    data=url_clean.sub(r'',data)
    return data

def remove_html(data):
    data = re.sub(r"https?://[^\s]+", "", data)
    
    return data

def remove_url(data):
    url_clean= re.compile(r"https://\S+|www\.\S+")
    data=url_clean.sub(r'',data)
    return data

def remove_stopwords(df, column):
    stop_words = set(stopwords.words("english"))
    stop_words.add("see")
    
    def remove_stopwords_from_text(text):
        word_tokens = word_tokenize(text)
        filtered_tokens = [token for token in word_tokens if not token.lower() in stop_words]
        return " ".join(filtered_tokens)
    
    df[column] = df[column].apply(remove_stopwords_from_text)
    return df

def clean_text_linkedIn(text):
    text = clean(text ,no_urls = True ,no_emoji=True, no_emails = True, lower = False, no_punct = False, no_currency_symbols=True, no_phone_numbers=True, replace_with_url="", replace_with_currency_symbol="", replace_with_email="", replace_with_phone_number="")
    return text

def convert_lowercase(text):
    return clean(text, lower = True)

# df2_cleaned_txt["text"] = df2_cleaned_txt["text"].apply(remove_hashtags)
# df2_cleaned_txt["text"] = df2_cleaned_txt["text"].apply(clean_text_linkedin)
# df2_cleaned_txt = remove_stopwords(df2_cleaned_txt, "text")

In [3]:
df2_sent = pd.read_csv("/Users/sreevaatsav/Desktop/Data_merge/deduped_linkedin_feb25.csv")

In [4]:
df2_sent.shape

(28519, 7)

In [5]:
df2_sent["text"] = df2_sent["text"].apply(remove_hashtags)
df2_sent["text"] = df2_sent["text"].apply(remove_comma)
df2_sent["text"] = df2_sent["text"].apply(remove_html)
df2_sent["text"] = df2_sent["text"].apply(remove_punctuations)
df2_sent["text"] = df2_sent["text"].apply(clean_text_linkedIn)
df2_sent["text"] = df2_sent["text"].apply(convert_lowercase)

In [6]:
def get_emotions(df):
    
    emotions_arr = []
    
    for i in range(len(df)):
        
        try:
            classes = emotion_clf(df.iloc[i]["text"])[0]
            temp_arr = []
            for cls_dic in classes:
    #             print(cls_dic)
                temp_arr.append(cls_dic["score"])
    #         print()
            emotions_arr.append(temp_arr)
            
        except:
            emotions_arr.append([None]*7)


    return emotions_arr

In [7]:
from transformers import pipeline

In [8]:
emotion_clf = pipeline("text-classification", model="j-hartmann/emotion-english-distilroberta-base", return_all_scores=True)

/Users/sreevaatsav/.pyenv/versions/project_env/lib/python3.10/site-packages/transformers/pipelines/text_classification.py:104: UserWarning: `return_all_scores` is now deprecated,  if want a similar funcionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [9]:
import time 

s = time.time()
emotions_arr = get_emotions(df2_sent)
e = time.time()
print((e-s)/60, " mins")

Token indices sequence length is longer than the specified maximum sequence length for this model (535 > 512). Running this sequence through the model will result in indexing errors


44.24813131491343  mins


In [13]:
# emotions_arr
df2_emotions = df2_sent.copy()
sent_df_obtained = pd.DataFrame(emotions_arr, columns=['anger', 'disgust', 'fear', 'joy', 'neutral', 'sadness', 'surprise'])
df2_emotions = pd.concat([df2_emotions, sent_df_obtained], axis=1)

In [15]:
del df2_emotions["Unnamed: 0"]

In [18]:
import os

os.listdir()

['tweets_merged_feb25.csv',
 'locations_tweets.csv',
 '.DS_Store',
 'requirements.txt',
 'Untitled.ipynb',
 '25.html',
 'recession_tweets.csv',
 'anger_posts',
 'cities1000.zip',
 'Temp.ipynb',
 'emotions_1_linkedin.csv',
 '.gitignore',
 'temp.py',
 'anger_posts.html',
 'world-countries.json',
 '.python-version',
 '.ipynb_checkpoints',
 'Analysis_vats.ipynb',
 'cities1000.txt',
 'world.json',
 'us-states.json',
 '.git',
 'linkedin_merged.csv',
 'py-googletrans',
 'Analysis.ipynb',
 '25']

In [19]:
df2_emotions.to_csv("emotions_2_linkedin.csv")

In [20]:
df1 = pd.read_csv("emotions_1_linkedin.csv")

In [22]:
del df1["Unnamed: 0.1"]
del df1["Unnamed: 0"]

In [23]:
df1

,text,job,date,num_reacts,num_cmts,num_rpsts,anger,disgust,fear,joy,neutral,sadness,surprise
0,i avoided publicizing my recent layoff in effo...,\n Global Speaker & Corporate Train...,2022-01-12,55,16,12,0.846311,0.001503,0.010997,0.015749,0.000652,0.110448,0.014341
1,well my feed is layoff after layoff my heart w...,\n Senior Client Partner | #paid\n ...,2022-01-12,33,3,0,0.001665,0.000214,0.000630,0.983621,0.000776,0.010972,0.002122
2,after 10 years and change at bleacher report i...,\n #Recruiter #talentacquisition\n ...,2022-01-12,18,8,0,0.005517,0.001246,0.001343,0.185619,0.009359,0.782184,0.014732
3,it has officially been 1 month since i was a p...,\n Software Development Engineer\n ...,2022-01-12,63,4,0,0.354407,0.001208,0.307034,0.299094,0.003856,0.030237,0.004164
4,layoffs happening left and right in the rest o...,\n Data Scientist at DoorDash\n ...,2022-01-12,157,2,0,0.036426,0.003197,0.189195,0.241905,0.084070,0.382541,0.062666
...,...,...,...,...,...,...,...,...,...,...,...,...,...
40366,a todos mis amigos y colegas de newell brands ...,"\n Manager, Talent Coordination at ...",2023-01-28,0,0,0,0.055489,0.008895,0.057691,0.044066,0.761159,0.035614,0.037085
40367,to all of my colleagues impacted by the google...,\n Brand Manager / Product Manager ...,2023-01-28,0,0,0,0.001423,0.000325,0.001082,0.991165,0.000929,0.004427,0.000649
40368,bdd tkn k n kr s laid off ud ym t k aag bddh n...,\n Product Design (Automation & AI)...,2023-01-28,1,0,0,0.224345,0.006282,0.034109,0.362575,0.077115,0.241151,0.054424
40369,tropic is hiring sales development reps apply ...,\n Sales Development Representative...,2023-01-28,0,0,0,0.005968,0.001669,0.005390,0.664069,0.304817,0.007279,0.010807


In [25]:
df_comb = pd.concat([df1, df2_emotions], axis=0)

In [27]:
df_comb.to_csv("emotions_linkedin_feb26.csv")

In [33]:
df_comb[df_comb["anger"].isna()]

,text,job,date,num_reacts,num_cmts,num_rpsts,anger,disgust,fear,joy,neutral,sadness,surprise
22,vc funding in vr ar virtual world has dropped ...,\n Technical Recruiter #OpenToWork\...,2022-01-12,3,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
48,it s with a very heavy heart that i m announci...,\n Untangling You from Confusion an...,2022-01-12,33,16,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
105,like many others i resigned from twitter a few...,\n Helping you automate contracts s...,2022-01-12,70,33,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
150,after the recent takeover of twitter by elon m...,\n Student @MIT '25 | Consulting | ...,2022-01-12,10,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
198,the recent layoffs in the usa have impacted a ...,\n Technical University Recruiter a...,2022-01-12,2146,105,16,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
28358,jeffrey pfeffer the thomas d dee ii professor ...,"\n Championing game makers, thinkin...",2023-02-26,35,5,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28360,long post alert i ve gotten a lot of new conne...,\n Former Technical Designer at Gen...,2023-02-26,39,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28369,after being impacted by a mass 3 months ago hu...,\n Founder @ new 🚀 + Legal Eagle Ve...,2023-02-26,19,22,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28461,during this layoffs horrible situation don t s...,\n Strategic Planning for Developin...,2023-02-26,1,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
df2_emotions[df2_emotions["anger"]]

,text,job,date,num_reacts,num_cmts,num_rpsts,anger,disgust,fear,joy,neutral,sadness,surprise
18,its been a difficult week even as a googler no...,\n MBA ( MARKETING &HR )||Digital m...,2023-01-29,1,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35,coegeun big tech hoesadeulyi layoff sataereul ...,\n Co Founder at Ringle 1:1 Online ...,2023-02-09,787,12,28,NaN,NaN,NaN,NaN,NaN,NaN,NaN
86,does anyone know of anyone hiring in tyler are...,"\n 12,314 followers\n",2023-01-29,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
175,fired relax you re good it s a day like any ot...,\n We’re hiring @ Checkr!\n,2023-01-29,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
199,robert kiyosaki says we are in global recessio...,"\n 2019 IDG ASEAN Top50 CIO, into S...",2023-01-29,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
28358,jeffrey pfeffer the thomas d dee ii professor ...,"\n Championing game makers, thinkin...",2023-02-26,35,5,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28360,long post alert i ve gotten a lot of new conne...,\n Former Technical Designer at Gen...,2023-02-26,39,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28369,after being impacted by a mass 3 months ago hu...,\n Founder @ new 🚀 + Legal Eagle Ve...,2023-02-26,19,22,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28461,during this layoffs horrible situation don t s...,\n Strategic Planning for Developin...,2023-02-26,1,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
